# Digital Fourier Transform (DFT) exploration
Udpated - 6 Nov. 2020

**NOTE** This code is only used as exploration, it is not part of any of the hackathon analysis or figure generation.

Exploring using DFT on the multiplex images for potential registration downstream.

High pass filter is edge detection --> image returns edges

Low pass filter is a detail detection but losses edges --< image returns the original image but blurred

In [ ]:
import sys
sys.path.append('..')

from mpleximreg.utils import parse_tif_dir, normalize_image
import cv2
from imageio import imread
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np

rcParams['figure.figsize'] = (10, 10)

In [ ]:
# get the filepaths to the images to use
filepaths = parse_tif_dir('/data/breastCancer/', quench=True)

# select the DAPI image, channel 2 image, and quench image to look at
dapi_im = normalize_image(imread(filepaths[1][1]))
ch2_im = normalize_image(imread(filepaths[1][2]))
quench_im = normalize_image(imread(filepaths['5Q'][1]))

In [ ]:
# calculate the fft of all three images and view
def calculate_fft(im):
    dft = cv2.dft(np.float32(im),flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
    return dft_shift, magnitude_spectrum


def lp_filter(img, threshold=30):
    # blurs images
    dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    
    rows, cols = img.shape
    crow,ccol = rows//2 , cols//2

    # create a mask first, center square is 1, remaining all zeros
    mask = np.zeros((rows,cols,2),np.uint8)
    mask[crow-threshold:crow+threshold, ccol-threshold:ccol+threshold] = 1

    # apply mask and inverse DFT
    fshift = dft_shift*mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])
    return img_back


def hp_filter(img, threshold=30):
    # edge detection
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    
    rows, cols = img.shape
    crow,ccol = rows//2 , cols//2
    
    fshift[crow-threshold:crow+threshold, ccol-threshold:ccol+threshold] = 0
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return img_back

In [ ]:
# display the raw images
fig, ax = plt.subplots(ncols=3, figsize=(20,10))
ax[0].imshow(dapi_im, cmap='gray')
ax[0].set_title('DAPI', fontsize=18)
ax[1].imshow(ch2_im, cmap='gray')
ax[1].set_title('Ch. 2', fontsize=18)
ax[2].imshow(quench_im, cmap='gray')
ax[2].set_title('Ch. 0', fontsize=18)
plt.show()

## Low Pass filtering -- blurring images

In [ ]:
shift = 30
dapi_filtered = lp_filter(dapi_im, threshold=shift)
ch2_filtered = lp_filter(ch2_im, threshold=shift)
quench_filtered = lp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

In [ ]:
shift = 40
dapi_filtered = lp_filter(dapi_im, threshold=shift)
ch2_filtered = lp_filter(ch2_im, threshold=shift)
quench_filtered = lp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

In [ ]:
shift = 20
dapi_filtered = lp_filter(dapi_im, threshold=shift)
ch2_filtered = lp_filter(ch2_im, threshold=shift)
quench_filtered = lp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

## High Pass filtering -- edge detection

In [ ]:
shift = 40
dapi_filtered = hp_filter(dapi_im, threshold=shift)
ch2_filtered = hp_filter(ch2_im, threshold=shift)
quench_filtered = hp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

In [ ]:
shift = 30
dapi_filtered = hp_filter(dapi_im, threshold=shift)
ch2_filtered = hp_filter(ch2_im, threshold=shift)
quench_filtered = hp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

In [ ]:
shift = 60
dapi_filtered = hp_filter(dapi_im, threshold=shift)
ch2_filtered = hp_filter(ch2_im, threshold=shift)
quench_filtered = hp_filter(quench_im, threshold=shift)

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_filtered, cmap='gray')
ax[1].imshow(ch2_filtered, cmap='gray')
ax[2].imshow(quench_filtered, cmap='gray')
plt.show()

In [ ]:
dapi_fft = calculate_fft(dapi_im)[1]
ch2_fft = calculate_fft(ch2_im)[1]
quench_fft = calculate_fft(quench_im)[1]

fig, ax = plt.subplots(ncols=3, figsize=(15,10))
ax[0].imshow(dapi_fft, cmap='gray')
ax[1].imshow(ch2_fft, cmap='gray')
ax[2].imshow(quench_fft, cmap='gray')
plt.show()